In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Document, Text, Integer
from elasticsearch.helpers import bulk
import numpy as np

In [2]:
df = pd.read_csv('data_youtube.csv',delimiter=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1477 entries, 0 to 1476
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Jeu                       1477 non-null   object
 1   Titre                     1477 non-null   object
 2   Chaîne                    1477 non-null   object
 3   Nombre de vues            1477 non-null   int64 
 4   Date de la mise en ligne  1477 non-null   object
dtypes: int64(1), object(4)
memory usage: 57.8+ KB


In [3]:
df2 = pd.concat([df['Titre'],df['Nombre de vues']],axis=1)
df2 = df2.drop_duplicates()
df2

,Titre,Nombre de vues
0,Trevor Didn't Forget 💀 || GTA V,18000000
1,Completing the most painful dares in GTA 5,772000
2,Which car can climb to the highest level in GT...,1000000
3,Robbing a MOPAR DEALERSHIP in GTA 5 RP!,434000
4,SHORTEST vs TALLEST PERSON in GTA 5!,654000
...,...,...
1472,DragonFlight Gameplay. Unboxing Unclaimed Blac...,129
1473,Allison's World of Warcraft Stream,53
1474,I wish I found a girl in World Of Warcraft,339
1475,Level With Me (Part 4) || World of Warcraft,41


In [4]:
data = df.to_dict('records')

In [5]:
data

[{'Jeu': 'Grand Theft Auto V',
  'Titre': "Trevor Didn't Forget 💀 || GTA V",
  'Chaîne': 'HigHub',
  'Nombre de vues': 18000000,
  'Date de la mise en ligne': 'il y a 5 jours'},
 {'Jeu': 'Grand Theft Auto V',
  'Titre': 'Completing the most painful dares in GTA 5',
  'Chaîne': 'GrayStillPlays',
  'Nombre de vues': 772000,
  'Date de la mise en ligne': 'il y a 6 jours'},
 {'Jeu': 'Grand Theft Auto V',
  'Titre': 'Which car can climb to the highest level in GTA 5?',
  'Chaîne': 'GrayStillPlays',
  'Nombre de vues': 1000000,
  'Date de la mise en ligne': 'il y a 2 jours'},
 {'Jeu': 'Grand Theft Auto V',
  'Titre': 'Robbing a MOPAR DEALERSHIP in GTA 5 RP!',
  'Chaîne': 'FuriousFade',
  'Nombre de vues': 434000,
  'Date de la mise en ligne': 'il y a 6 jours'},
 {'Jeu': 'Grand Theft Auto V',
  'Titre': 'SHORTEST vs TALLEST PERSON in GTA 5!',
  'Chaîne': 'Caylus',
  'Nombre de vues': 654000,
  'Date de la mise en ligne': 'il y a 4 jours'},
 {'Jeu': 'Grand Theft Auto V',
  'Titre': 'GTA V But 

In [6]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200,"scheme": "http"}])

In [7]:
es.ping()

True

In [8]:
def generate_data(data):
    for docu in data:
        yield {
            "_index": "yt_twitch",
            "_type": "video",
            "_source": {k:v if v else None for k,v in docu.items()},
        }
        
if es.indices.exists('yt_twitch')==True:
    es.indices.delete(index='yt_twitch')
    bulk(es, generate_data(data))
else :
    bulk(es, generate_data(data))

C:\Users\vgvic\AppData\Local\Temp\ipykernel_6884\2793155111.py:9: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists('yt_twitch')==True:
c:\Users\vgvic\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [9]:
QUERY={"query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "bool": {
            "should": [
              {
                "match": {
                  "Jeu.keyword": "League of Legends"
                }
              }
            ],
            "minimum_should_match": 1
          }
        }
      ],
      "should": [],
      "must_not": []
    }
  },}

  

In [10]:
result = es.search(index="yt_twitch", body=QUERY, size = 1500)

C:\Users\vgvic\AppData\Local\Temp\ipykernel_6884\3229683340.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="yt_twitch", body=QUERY, size = 1500)


In [11]:
[elt['_source'] for elt in result["hits"]["hits"]]

[]

In [12]:
result = es.search(index="yt_twitch", body={"query": {"match_all": {}}},size = 1500)
result

C:\Users\vgvic\AppData\Local\Temp\ipykernel_6884\3116974301.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="yt_twitch", body={"query": {"match_all": {}}},size = 1500)


{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [15]:
QUERY ={
"query": {
    "bool": {
    "must": [],
    "filter": [
        {
        "bool": {
            "should": [
            {
                "match_phrase": {
                "Jeu" : "Apex Legends"
                }
            }
            ],
            "minimum_should_match": 1
        }
        }
    ],
    "should": [],
    "must_not": []
    }
}
}
result = es.search(index="yt_twitch", body=QUERY,size=1500)

filtre_jeux = []
[filtre_jeux.append(elt['_source']) for elt in result["hits"]["hits"]]
filtre_jeux

C:\Users\vgvic\AppData\Local\Temp\ipykernel_6884\2960689429.py:24: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="yt_twitch", body=QUERY,size=1500)


[{'Jeu': 'Apex Legends',
  'Titre': 'THE NEMESIS IS INSANE',
  'Chaîne': 'iiTzTimmy',
  'Nombre de vues': 388000,
  'Date de la mise en ligne': 'il y a 4 jours'},
 {'Jeu': 'Apex Legends',
  'Titre': 'USING MOVEMENT TO KILL TWITCH STREAMERS IN APEX LEGENDS #14',
  'Chaîne': 'Faide',
  'Nombre de vues': 336000,
  'Date de la mise en ligne': 'il y a 4 jours'},
 {'Jeu': 'Apex Legends',
  'Titre': 'Movement Player Uses The Nemesis Burst AR',
  'Chaîne': 'Faide',
  'Nombre de vues': 300000,
  'Date de la mise en ligne': 'il y a 6 jours'},
 {'Jeu': 'Apex Legends',
  'Titre': 'THE NEW R-99 BUFF + MOVEMENT PLAYS',
  'Chaîne': 'Faide',
  'Nombre de vues': 288000,
  'Date de la mise en ligne': 'il y a 3 jours'},
 {'Jeu': 'Apex Legends',
  'Titre': '6K DAMAGE IN SOLO VS SQUADS',
  'Chaîne': 'Faide',
  'Nombre de vues': 263000,
  'Date de la mise en ligne': 'il y a 5 jours'},
 {'Jeu': 'Apex Legends',
  'Titre': 'MY FIRST RANKED GAME IN SEASON 16',
  'Chaîne': 'iiTzTimmy',
  'Nombre de vues': 233000